Download Code

In [1]:
#Download Code
import os
path = "/content/"
os.chdir(path)
!rm -rf ../NeuralNLP-NeuralClassifier
!git clone https://github.com/ComputationalResearchProjects/NeuralNLP-NeuralClassifier
path = "/content/NeuralNLP-NeuralClassifier/"
os.chdir(path)

Cloning into 'NeuralNLP-NeuralClassifier'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 136 (delta 26), reused 14 (delta 14), pack-reused 90
Receiving objects: 100% (136/136), 13.66 MiB | 6.34 MiB/s, done.
Resolving deltas: 100% (57/57), done.


Train Model

In [ ]:
import json
import random
import networkx as nx
random_seed = 2023
# Upload bert_HC2.py to ./NeuralNLP-NeuralClassifier/
import bert_HC2 as f
import imp
imp.reload(f)

# !mkdir data
#Upload varia_label_dict2.json to ./NeuralNLP-NeuralClassifier/data/
!mkdir data1

data_dict = f.read_json("data/varia_label_dict2.json")
train_data, dev_data, test_data, node_list = f.build_train_dev_test(data_dict, random_seed)

f.write_json(train_data, "data1/train.json")
f.write_json(dev_data, "data1/dev.json")
f.write_json(test_data, "data1/test.json")
f.write_taxonomy(node_list, "data1/taxonomy.txt")

!python train.py conf/train_new.json
!python predict.py conf/train_new.json data1/test.json 
!cp -r predict.txt 3_train_test_predictions2.txt

Compute Accuracy

In [ ]:
import bert_HC2 as f
import imp
imp.reload(f)
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import confusion_matrix

# Input
test_pred_label_list = []
file = open("3_train_test_predictions2.txt", "r")
for line in file:
    test_pred_label_list.append(line.strip("\n"))
file.close()

#Process
test_label_list = []
for item in test_data:
    test_label_list.append(item["doc_label"][0])
var_list = []
for item in test_data:
    var_list.append(" ".join(item["doc_token"]))

print("Classification Report:\n", classification_report(test_label_list, test_pred_label_list))

t_label_list = []
for item in test_label_list:
    if item not in t_label_list:
        t_label_list.append(item)

t_matrix = confusion_matrix(test_label_list, test_pred_label_list, labels = t_label_list)
df = pd.DataFrame(t_matrix, columns = t_label_list, index = t_label_list)
df.to_csv("temp.csv")


Use Model to Extract Info Automatically

In [5]:
imp.reload(f)
data_pred_dict = f.read_json("data/varia_label_pred_dict.json")
test_data2 = f.build_test_pred(data_pred_dict)
f.write_json(test_data2, "data1/test2.json")

!python train.py conf/train_new.json
!python predict.py conf/train_new.json data1/test2.json 
!cp -r predict.txt 3_test_predictions2.txt
!cp -r predict_pro.txt 3_predict_pro.txt

Use dataset to generate dict.
Size of doc_label dict is 210
Size of doc_token dict is 1778
Size of doc_char dict is 73
Size of doc_token_ngram dict is 0
Size of doc_keyword dict is 0
Size of doc_topic dict is 0
Shrink dict over.
Size of doc_label dict is 210
Size of doc_token dict is 1655
Size of doc_char dict is 72
Size of doc_token_ngram dict is 0
Size of doc_keyword dict is 0
Size of doc_topic dict is 0
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Train performance at epoch 1 is precision: 0.108874, recall: 0.108874, fscore: 0.108874, macro-fscore: 0.003921, right: 146, predict: 